In [6]:
# Save Model Using Pickle
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)
# Fit the model on training set
model = LogisticRegression()
model.fit(X_train, Y_train)
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
%%writefile logistic_model_service.py
import pandas as pd
from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.service.artifacts.common import PickleArtifact

@env(pip_packages=["sklearn"])
@artifacts([PickleArtifact('model')])
class LogisticModel(BentoService):
    """
    A minimum prediction service exposing a Scikit-learn model
    """

    @api(input=DataframeInput(), batch=True)
    def predict(self, df: pd.DataFrame):
        """
        An inference API named `predict` with Dataframe input adapter, which codifies
        how HTTP requests or CSV files are converted to a pandas Dataframe object as the
        inference API function input
        """
        return self.artifacts.model.predict(df)

Overwriting logistic_model_service.py


In [9]:
#import pickle to load dumped model
import pickle

# import the LogisticModel class defined above
from logistic_model_service import LogisticModel


# import dumped model
filename = 'finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))


# Create a logistic model service instance
logistic_model_service = LogisticModel()

# Pack the newly trained model artifact
logistic_model_service .pack('model', loaded_model )

# Save the prediction service to disk for model serving
saved_path = logistic_model_service .save()

[2021-07-09 12:17:05,979] INFO - BentoService bundle 'LogisticModel:20210709121705_8D9D86' saved to: /home/ubuntu/bentoml/repository/LogisticModel/20210709121705_8D9D86


In [1]:
!bentoml serve LogisticModel:latest --port 5000

[2021-07-09 11:29:29,645] INFO - Getting latest version LogisticModel:20210709112532_463C04
[2021-07-09 11:29:29,654] INFO - Starting BentoML API proxy in development mode..
[2021-07-09 11:29:29,655] INFO - Starting BentoML API server in development mode..
[2021-07-09 11:29:29,783] INFO - Micro batch enabled for API `predict` max-latency: 20000 max-batch-size 4000
[2021-07-09 11:29:29,783] INFO - Your system nofile limit is 4096, which means each instance of microbatch service is able to hold this number of connections at same time. You can increase the number of file descriptors for the server process, or launch more microbatch instances to accept more concurrent connection.
======== Running on http://0.0.0.0:5000 ========
(Press CTRL+C to quit)
 * Serving Flask app 'LogisticModel' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:42035/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jul/2021 11:34:21] "POST 

In [12]:
!bentoml containerize LogisticModel:latest -t logistic-model:latest

[2021-07-09 12:19:45,974] INFO - Getting latest version LogisticModel:20210709112532_463C04
Found Bento: /home/ubuntu/bentoml/repository/LogisticModel/20210709112532_463C04
Containerizing LogisticModel:20210709112532_463C04 with local YataiService and docker daemon from local environment/Build container image: logistic-model:latest


In [ ]:
!docker run -p 5000:5000 logistic-model:latest --workers=1 --enable-microbatch